<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Open Source Decision Intelligence

Decision Intelligence applied in this module:  
* Listing key factors to consider when making a sound decision  
* Using OSS (open-source) reasoning models to optimize the decision approach 
* Decision Scenario: Use a decision framework (Ben Franklin's Pro & Con List) to create a decision plan  
* Improving Decision Intelligence process by explicitly proving decision frameworks and additional context 

A recommended enterprise pattern is to scale Articial Intelligence strategy with three key areas:
* Commercial AI (OpenAI and other proprietary Generative AI providers)
* Open-Source AI (open-source AI providers)
* Vendor and Partner AI (i.e. company HR Software, contract software) 

These three areas together strategically form AI capability and capacity in what I like to refer to as the "Generative AI Brain". This is illustrated below with sample providers. For example, an organization may have an existing relationship with Adobe for their graphic design and leverage the built-in Generative AI capabilities rather building their own. 

<img style="display: block; margin: auto;" width ="800px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/AIBrainPillars.png">

Semantic Kernel embraces AI orchestration across all the pillars mentioned above. It allows all types of models (commercial or proprietary) and almost any APIs to be orchestrated to faciliate enterprise Decision Intelligence.

### Step 1 - Get Started with LMStudio and Local Open Source AI Models 

This module highlights how to use local GenAI models (i.e. LLMs) with Semantic Kernel. To illustrate this a model from the Phi-4 family will be selected to run locally using LMStudio as a local REST endpoint that will interface with Semantic Kernel orchestration.  

The Phi-4 family of models are considered an SLMs (Small Language Models) with parameters generally below the 27 Billion parameter threshhold. This doesn't mean these models are not capable of performing well. While it certainly may lack the breadth of general knowledge of Large Language Models, Phi-4 models make up for it by providing very capable logic, math and reasoning capabilities that can rival frontier LLMs. Below is an image illustrating this performance:  
<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Phi4-Performance.webp">  

The Phi-4 family of models doesn't include just general purpose capabilities. In April 2025, the Phi-4 family introduced specialized reasoning models. These reasoning models have an "inner monologue" that asks additional further qualifying questions before returning an answer. 

Below is a data visualization of the performance of selected Phi-4-Reasoning models. The accuracy distribution illustrates the variance in performance from 50 independent runs from the AIME 2025 problem solving data set (https://artofproblemsolving.com/wiki/index.php/2025_AIME_I). Notice that the base Phi-4-Reasoning model and the Phi-4-Reasoning-Plus model offer an accuracy the approaches OpenAI's o3-mini model, which up until April 2025 was one the best performing reasoning models available commercially. This has significant implications as Phi-4-Reasoning models are only 14 billion parameters and can be run on simple commodity (gaming workstation, laptops with GPUs) hardware!  
<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Phi4-Reasoning-Benchmark.jpg">  


The bottom line is the small size, reasoning performance makes Phi-4 reasoning models ideal for Decision Intelligence with open-source AI, where the quality of the approach is critical to great decision outcomes. 

Steps to get started:
* Download & install the latest LMStudio version: https://lmstudio.ai/ (Windows, Mac or Linux) 
* Run the LMStudio studio application 
* In the LMStudio application, download the Phi-4-Reasoning GGUF file. Search for "Phi-4-Reasoning" in the "Discover" section of LMStudio. A variety of Phi-4-Reasoning options that are official and unofficial from hobbyists will appear. Typically, selecting the model with the most downloads will provide the best results. You can select different quantizations of the model, to optimize the performance  
* In the experiment below, the 14B parameter model is being used. LMStudio will inspect your hardware and let you know which quantized version of the model(s) is optimal for your hardware. Note: Even computers with small graphics cards can run these models well locally. Furthermore, laptops such as the Macbook Pro with Neural Engine can run LMStudio local models as well. 
* Start the LMStudio Server with the Phi-4 model loaded. This will start a local REST endpoint with a URI similar to http://localhost:1234/v1 
* The LMStudio local server does not have default security, you can simply check by navigating to this link in any browser to check if a model is loaded: http://localhost:1234/v1/models 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/LMStudioServer.png"> 

### Step 2 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next cell to:
* Use the Configuration Builder to use the local LMStudio Server  
* Use the API configuration to build the Semantic Kernel orchestrator  
* Notice there is no security being passed in and it is simply a URL

In [1]:
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.4"
#r "nuget: Microsoft.SemanticKernel, 1.48"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

#pragma warning disable SKEXP0010 
var semanticKernel = Kernel.CreateBuilder()
    .AddOpenAIChatCompletion(
        modelId: "phi-4-reasoning", // Put in the name of your model i.e. qwen3-30b-a3b
        endpoint: new Uri("http://localhost:1234/v1/"),
        apiKey: null)
    .Build();
#pragma warning restore SKEXP0010

// Retrieve the Chat Completion Service
var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 9.0.4 Microsoft.SemanticKernel, 1.48.0

### Step 3 - Open Source AI Models with Decision Intelligence 

Semantic Kernel allows one to interact with any API service that adheres to the OpenAI specification. Notice the method to add LMStudio capability was simply enabled via the **AddOpenAIChatCompletion** method above. 

Execute the cell below about decision factors for a investment property. Note:
* OpenAI Prompt Execution Settings are the same in LMStudio as they are for OpenAI and Azure OpenAI
* OSS models have specific model cards identifying best practices 
* OSS reasoning models will output their reasoning tokens inside the <think> XML tags
* Passing in arguents works the same way in Semantic Kernel as other models 
* Streaming works as well in Semantic Kernel for supported services 

> 📜 "Don't wait to buy real estate. Buy real estate and wait."  
>
> -- <cite>Will Rodgers (American humorist, actor, and social commentator in the early 20th century)</cite>  

In [17]:
// Define the system prompt for the Decision Intelligence assistant
var systemDecisionPrompt = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Create a Decision Intelligence prompt on the topic of purchasing a secondary home as an investment property
// Provide detailed decision-making criteria for evaluating the investment decision
var simpleDecisionPrompt = """
You are considering purchasing a secondary home as an investment property. 

What key factors should you evaluate to ensure a sound investment decision, including financial, 
market, and property-specific considerations? 
Outline the critical steps and criteria for assessing location, potential rental income, 
financing options, long-term property value, and associated risks. 
""";

// Note: Different OSS models have different capabilities and settings
// Phi-4 Model Card Recommendations: https://huggingface.co/microsoft/Phi-4-reasoning 
// Note 2: The settings below are similar to OpenAI API settings. Most OSS models support these settings  
// Note 3: The reasoning plus models are designed to be much more verbose in their thinking. Extra tokens will be required! 
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 6000, 
    Temperature = 0.8, 
    TopP = 0.95,
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemDecisionPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory, openAIPromptExecutionSettings))
{
    // Clearly seperate the thinking process from the final answer
    var contentString = content.ToString().Replace(@"</think>", 
        @"</think>" + Environment.NewLine + "--------------------------" + Environment.NewLine + Environment.NewLine);
    chatResponse += contentString;
    Console.Write(contentString);
}

// Add the assistant's response to the chat history
// This allows you to maintain the context of the conversation for future messages
chatHistory.AddAssistantMessage(chatResponse);

<think>User prompt: "You are considering purchasing a secondary home as an investment property. What key factors should you evaluate to ensure a sound investment decision, including financial, market, and property-specific considerations? Outline the critical steps and criteria for assessing location, potential rental income, financing options, long-term property value, and associated risks."

This is question about decision intelligence regarding purchasing a secondary home as an investment property. Our answer: We'll provide structured answer with headings on various factors such as financial considerations, market conditions, property-specific evaluations, etc.

I need to follow guidelines: "You are Phi" but I'm not required to mention disclaimers at the beginning and end for medical etc. But instructions say "provide disclaimer both at beginning and end if topics sensitive." Wait check instructions: "When replying topics above at every message" Actually guidelines said: "provide a 

Advanced Prompt Engineering techniques can be applied to OSS (open-source) models as well. In the example below a more advanced reasoning decision prompt will be used to provide additional instructions to the GenAI model. Reasoning models do a nice job in approaching the problem with an inner monologue, however you can provide additional instructions for the model to consider as they are thinking about an approach.  

In [18]:
// Define the system prompt for the Decision Intelligence assistant
var systemDecisionPrompt = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// Create a Decision Intelligence prompt on the topic of purchasing a secondary home as an investment property
// Use Chain of Thought to prompt the OSS model
// Use the Minto Pyramid to communicate the decision 
var advancedDecisionPrompt = """
You are considering purchasing a secondary home as an investment property. 

Before providing any answer, in your reasoning process consider the following:
Understand the Problem: Carefully read and understand the user's question or request. 
Break Down the Reasoning Process: Outline the steps required to solve the problem or respond to the request logically and sequentially. Think aloud and describe each step in detail. 
Always aim to make your thought process transparent and logical. 
Explain Each Step: Provide reasoning or calculations for each step, explaining how you arrive at each part of your answer. 
Provide structured, logical, and comprehensive advice. 
Arrive at the Final Answer: Only after completing all steps, provide the final answer or solution. 
Review the Thought Process: Double-check the reasoning for errors or gaps before finalizing your response. 
Communicate the final decision using the Minto Pyramid Principle.
""";

// Note: Different OSS models have different capabilities and settings
// Phi-4-Reasoning Model Card Recommendations: https://huggingface.co/microsoft/Phi-4-reasoning 
// Note 2: The settings below are similar to OpenAI API settings. Most OSS models support these settings 
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 6000, 
    Temperature = 0.8, 
    TopP = 0.95,
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// Create a new chat history object with proper system and user message roles
var chatHistoryAdvancedDecisionPrompt = new ChatHistory();
chatHistoryAdvancedDecisionPrompt.AddSystemMessage(systemDecisionPrompt);
chatHistoryAdvancedDecisionPrompt.AddUserMessage(advancedDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryAdvancedDecisionPrompt, openAIPromptExecutionSettings))
{
    // Clearly seperate the thinking process from the final answer
    var contentString = content.ToString().Replace(@"</think>", 
        @"</think>" + Environment.NewLine + "--------------------------" + Environment.NewLine + Environment.NewLine);
    chatResponse += contentString;
    Console.Write(contentString);
}

// Add the assistant's response to the chat history
// This allows you to maintain the context of the conversation for future messages
chatHistoryAdvancedDecisionPrompt.AddAssistantMessage(chatResponse);

<think>User: "You are considering purchasing a secondary home as an investment property." It seems like we have an instruction from the user asking to consider purchasing a secondary home as an investment property. The instructions: "Before providing any answer, in your reasoning process consider the following: ..." It instructs that we need to outline our steps. But the problem is ambiguous. There's no question? Possibly it might be an initial scenario question that expects us to explore options and provide advice.

The conversation text says: "You are considering purchasing a secondary home as an investment property." This seems like a prompt for decision intelligence. We must produce a message following instructions from our guidelines with clarity and transparency, including reasoning process steps. But careful note: the system instructs that we should not share chain-of-thought details. However instructions said "explain each step" but instructions say "Do not share private chain-

### Step 4 - Open Source AI Models with Ben Franklin Decision Framework 

> 📜 "By failing to prepare, you are preparing to fail." 
>
> -- <cite>Ben Franklin (Founding Father of the United States, inventor, godfather of Decision Science)</cite> 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/Articles/main/20230326-Make-Great-Decisions-Using-Ben-Franklins-Pros-And-Cons-Method/Image-BenFranklinDecisionMakingMethod.png">

#### Tom Brady's use of a Decision Framework

Tom Brady's decision to join the Tampa Bay Buccaneers in 2020 marked a significant in his legendary NFL career. After 20 seasons and six Super Bowl championships with the New England Patriots, Brady became a free agent and chose to sign with the Bucs. How did he arrive at this decision? On the Fox broadcast on 09.29.2024, while covering the Buccaneers vs Philadelphia Eagles game, Tom Brady described how he arrived at this decision.

In the screenshot below, Tom Brady is holding up some small paper cards he is showing the audience of the broadcast. Brady mentioned he wrote down the personal decision criteria that was important and how each team compared in that criteria (salary, weather etc). He used this to select the Tampa Bay Buccaneers as his team, where he went on to win a Super Bowl in his first year there! **After 250 years since it's inception, Tom Brady used the "Ben Franklin Decision Framework" to decide where to play NFL quaterback!!**  

> 📜 "There was a lot of reasons to choose Tampa, and I made about 18 criteria why, and there was things all the way from salary, obviously, to the weather to the facilities to how great the players were..."
>
> -- <cite>Tom Brady (7-time Super Bowl Champion, Hall of Fame NFL Quaterback)</cite> 

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/OpenSourceDecisionIntelligence-BenFranklinDecisionFramework-TomBrady.png">

#### Steps for Ben Franklin's Decision Framework

Below are the steps Ben Franklin recommends when making a decision, which he called his "Decision Making Method of Moral Algebra":  
- Frame a decision that has two options (Yes or a No)
- Divide an area into two competing halves: a "Pro" side and "Con" side
- Label the top of one side "Pro" (for) and the other "Con" (against)
- Under each respective side, over a period of time (Ben Franklin recommended days, this could be minutes) write down various reasons/arguments that support (Pro) or are against (Con) the decision
- After spending some time thinking exhaustively and writing down the reasons, weight the different Pro and Con reasons/arguments
- Determine the relative importance of each reason or argument. This is done by taking reasons/arguments that are of similar value (weight) and crossing them off of the other competing half. Multiple reasons can be combined from one side to form a "subjective" value (weight) to balance out the other half. (For example, two medium "Pro" reasons might add up to an equal value of a single important "Con" reason)
- The side with the most remaining reasons is the option one should select for the decision in question

Learn more about Ben Franklin's Decision Framework: https://medium.com/@bartczernicki/make-great-decisions-using-ben-franklins-decision-making-method-c7fb8b17905c  

#### Decision Scenario - Should a Family Take a Luxury Vacation

Should a family take a luxury family vacation this year? Just like Brady mapped out whether joining the Bucs would satisfy his key personal and professional goals, you can list the factors that matter most for your family—budget, timing, destination climate, activities for the kids—and lay them out on your own “decision cards.” Weigh each component carefully, just as Brady weighed his NFL future. Because if it worked to land Brady in Tampa (where he won yet another Super Bowl), imagine what it can do for a family’s dream getaway.

<img style="display: block; margin: auto;" width ="700px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/Scenarios/Scenario-Vacation.png">

In [19]:
// Define the system prompt for the Decision Intelligence assistant
var systemDecisionPrompt = """
You are a decision intelligence assistant. 
Your role is to guide users in exploring options, analyzing decisions, solving complex problems, and applying systems thinking to diverse scenarios. 
Provide structured, logical, and comprehensive advice, ensuring clarity, depth, and actionable insights to support informed decision-making.
""";
var benFranklinLuxuryVacationDecisionPrompt = """
Apply the Ben Franklin Decision-Making Framework (Pro and Con list) to evaluate whether or not to take a luxury family vacation. 
List at most 5 pros and at most 5 cons to help the user make an informed decision.
""";

var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 6000, 
    Temperature = 0.8, 
    TopP = 0.95
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// Create a new chat history object with proper system and user message roles
var chatHistoryLuxuryVacationDecision = new ChatHistory();
chatHistoryLuxuryVacationDecision.AddSystemMessage(systemDecisionPrompt);
chatHistoryLuxuryVacationDecision.AddUserMessage(benFranklinLuxuryVacationDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryLuxuryVacationDecision, openAIPromptExecutionSettings))
{
    // Clearly seperate the thinking process from the final answer
    var contentString = content.ToString().Replace(@"</think>", 
        @"</think>" + Environment.NewLine + "--------------------------" + Environment.NewLine + Environment.NewLine);
    chatResponse += contentString;
    Console.Write(contentString);
}

// Add the assistant's response to the chat history
// This allows you to maintain the context of the conversation for future messages
chatHistoryLuxuryVacationDecision.AddAssistantMessage(chatResponse);

<think>User request: "Apply the Ben Franklin Decision-Making Framework (Pro and Con list) to evaluate whether or not to take a luxury family vacation. List at most 5 pros and at most 5 cons to help the user make an informed decision." The instructions are to apply this framework.

The question is clear, no sensitive content. The guidelines: We have to include disclaimers if topic is medical, legal, etc. But here it's a family vacation evaluation; not sensitive topics. The request: "Apply Ben Franklin Decision-Making Framework" which involves listing pros and cons. I'll generate a list of up to 5 each.

I need to produce a structured answer that lists at most 5 pros and 5 cons for taking a luxury family vacation using the Ben Franklin decision-making framework. I have to provide both sides, then a conclusion. The instructions are "list at most 5 pros and at most 5 cons", so list exactly up to 5 each.

I'll produce a final answer that includes disclaimers if necessary? Possibly not requi

#### Improving the Ben Franklin's Decision Framework with SLMs

For those familiar with the Ben Franklin decision framework, the output from the AI model above may not be exactly what most would anticipate. The Ben Franklin framework may not be fully understood nor applied. Open-Source GenAI models that have a small amount of parameters (< ~14 billion parameters) may not have all the inherent Decision Intelligence knowledge "trained" into the model. The exception being domain-specific models that are only trained on data sets for that domain. Those domain-specific models can have a great deal of knowledge, while maintaining a small amount of parameters. Therefore, you could train small AI models that specialize in Decision Intelligence. 

One simple way to improve the outcome is to provide the explicit steps of the "Ben Franklin Decision Framework" into the prompt context. This basically provides the instructions of the decision framework directly to the model; regardless if the GenAI model was trained with decision framework data. By doing this extra explicit step, there is no ambiguity for the AI model how to approach the decision process. 

In the example below, the prompt context is provided with the Ben Franklin Decision Framework steps. Contrast this with the example above, where the decision recommendation is not clear. 

In [20]:
// Define the system prompt for the Decision Intelligence assistant
var systemDecisionPrompt = """
You are a decision intelligence assistant. 
Your role is to guide users in exploring options, analyzing decisions, solving complex problems, and applying systems thinking to diverse scenarios. 
Provide structured, logical, and comprehensive advice, ensuring clarity, depth, and actionable insights to support informed decision-making.
""";

var explicitBenFranklinDecisionPrompt = """
Apply the following steps IN ORDER of the Ben Franklin Decision Framework to the Question below:
1) Frame a decision that has two options (Yes or a No)
2) Divide an area into two competing halves: a "Pro" side and "Con" side
3) Label the top of one side "Pro" (for) and the other "Con" (against)
4) Under each respective side, list a maximum of 5 reasons or arguments for each option
5) Consider the weight of each reason or argument. This is done by taking reasons/arguments that are of similar value (weight) and crossing them off of the other competing half. Multiple reasons can be combined from one side to form a "subjective" value (weight) to balance out the other half. (For example, two medium "Pro" reasons might add up to an equal value of a single important "Con" reason)
6) Determine the relative importance of each reason or argument. This is done by taking reasons/arguments that are of similar value (weight) and crossing them off of the other competing half. Multiple reasons can be combined from one side to form a "subjective" value (weight) to balance out the other half. (For example, two medium "Pro" reasons might add up to an equal value of a single important "Con" reason)
7) The side with the most remaining reasons is the option one should select for the decision in question

IMPORTANT: ALWAYS recommend a decision based on the side with the most remaining reasons, even if the reasons are of lesser value than the other side!

Question: Should I take a luxury family vacation?
""";

// Note: Phi-4-Reasoning Model Card Recommendations: https://huggingface.co/microsoft/Phi-4-reasoning  
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 10000, 
    Temperature = 0.8, 
    TopP = 0.95
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// Create a new chat history object with proper system and user message roles
var chatHistoryLuxuryVacationDecisionUsingBenFranklinFramework = new ChatHistory();
chatHistoryLuxuryVacationDecisionUsingBenFranklinFramework.AddSystemMessage(systemDecisionPrompt);
chatHistoryLuxuryVacationDecisionUsingBenFranklinFramework.AddUserMessage(explicitBenFranklinDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryLuxuryVacationDecisionUsingBenFranklinFramework, openAIPromptExecutionSettings))
{
    // Clearly seperate the thinking process from the final answer
    var contentString = content.ToString().Replace(@"</think>", 
        @"</think>" + Environment.NewLine + "--------------------------" + Environment.NewLine + Environment.NewLine);
    chatResponse += contentString;
    Console.Write(contentString);
}

// Add the assistant's response to the chat history
// This allows you to maintain the context of the conversation for future messages
chatHistoryLuxuryVacationDecisionUsingBenFranklinFramework.AddAssistantMessage(chatResponse);

<think>User says: "You are Phi, a language model developed by Microsoft" but now it instructs that we should follow guidelines. Also instructions mention "Apply steps in order of Ben Franklin Decision Framework". Let's re-read instructions: The conversation is a request to apply the following steps in order of the Ben Franklin Decision Framework to the question below:

Steps:
1) Frame decision with two options (Yes or No)
2) Divide area into competing halves: Pro and Con side
3) Label top of one side "Pro" (for) and other "Con" (against)
4) Under each respective side, list maximum 5 reasons for each option.
5) Consider the weight of each reason. This is done by taking reasons/arguments that are of similar value (weight) and crossing them off the other competing half. Multiple reasons can be combined from one side to form a "subjective" value (weight) to balance out the other half. For example, two medium Pro reasons might add up to an equal value of a single important Con reason.
6) De

The GenAI model may or may not recommend a luxury vacation, but this can be dramatically improved further! Imagine if the GenAI model had access to: your finances, current stress level, the last time you took a vacation, any upcoming major purchases, family dynamic?!  

In the example below, additional context is provided with that information. Notice how it changes the the Pro and Con list.  

Just like Tom Brady, it could craft a Pro and Con list specific and personalized to your scenario!

In [21]:
// Define the system prompt for the Decision Intelligence assistant
var systemDecisionPrompt = """
You are a decision intelligence assistant. 
Your role is to guide users in exploring options, analyzing decisions, solving complex problems, and applying systems thinking to diverse scenarios. 
Provide structured, logical, and comprehensive advice, ensuring clarity, depth, and actionable insights to support informed decision-making.
""";
// Try changing the background information to see how it affects the decision-making process
var backroundInformation = """
BACKGROUND INFORMATION OF FAMILY: 
You are considering purchasing a secondary home as an investment property. 
You have been stressed out at work. 
You have been working long hours and have not taken a vacation in over a year. 
Your kids are about to enter college and they don't have scholarships. 
Your car is finishing its lease and will need to be replaced soon. 
""";
// Try to adjust the specificity of the decision-making criteria to see how it affects the decision-making process
var explicitBenFranklinDecisionPrompt = """
Apply the following steps IN ORDER of the Ben Franklin Decision Framework to the Question below:
1) Frame a decision that has two options (Yes or a No)
2) Divide an area into two competing halves: a "Pro" side and "Con" side
3) Label the top of one side "Pro" (for) and the other "Con" (against)
4) Under each respective side, list a maximum of 5 reasons or arguments for each option
5) Consider the weight of each reason or argument. This is done by taking reasons/arguments that are of similar value (weight) and crossing them off of the other competing half. Multiple reasons can be combined from one side to form a "subjective" value (weight) to balance out the other half. (For example, two medium "Pro" reasons might add up to an equal value of a single important "Con" reason)
6) Determine the relative importance of each reason or argument. This is done by taking reasons/arguments that are of similar value (weight) and crossing them off of the other competing half. Multiple reasons can be combined from one side to form a "subjective" value (weight) to balance out the other half. (For example, two medium "Pro" reasons might add up to an equal value of a single important "Con" reason)
7) The side with the most remaining reasons is the option one should select for the decision in question

IMPORTANT: ALWAYS recommend a decision based on the side with the most remaining reasons, even if the reasons are of lesser value than the other side!

Question: Should I take a luxury family vacation?
""";

// Note: Phi-4-Reasoning Model Card Recommendations: https://huggingface.co/microsoft/Phi-4-reasoning  
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 6000, 
    Temperature = 0.8, 
    TopP = 0.95
    };
KernelArguments kernelArguments = new KernelArguments(openAIPromptExecutionSettings);

// Create a new chat history object with proper system and user message roles
var chatHistoryLuxuryVacationDecisionWithBackground = new ChatHistory();
chatHistoryLuxuryVacationDecisionWithBackground.AddSystemMessage(backroundInformation);
chatHistoryLuxuryVacationDecisionWithBackground.AddUserMessage(backroundInformation);
chatHistoryLuxuryVacationDecisionWithBackground.AddUserMessage(explicitBenFranklinDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryLuxuryVacationDecisionWithBackground, openAIPromptExecutionSettings))
{
    // Clearly seperate the thinking process from the final answer
    var contentString = content.ToString().Replace(@"</think>", 
        @"</think>" + Environment.NewLine + "--------------------------" + Environment.NewLine + Environment.NewLine);
    chatResponse += contentString;
    Console.Write(contentString);
}

// Add the assistant's response to the chat history
// This allows you to maintain the context of the conversation for future messages
chatHistoryLuxuryVacationDecisionWithBackground.AddAssistantMessage(chatResponse);

<think>We are told the instructions. There's detailed guidelines from the user message, which is as follows:

"You are Phi, a language model developed by Microsoft, trained to provide accurate, secure, and user-aligned responses. Follow these principles to ensure clarity, safety, and ethical standards in all interactions." The instructions then contain steps on how to apply "the following steps IN ORDER of the Ben Franklin Decision Framework" to the question: "Should I take a luxury family vacation?" It is also provided background information for the family: purchasing secondary home as investment property; stressed out at work due to long hours and not taken vacation in over a year; kids are about to enter college with no scholarships available, car finishing lease. So we have additional context.

We need to apply "Ben Franklin Decision Framework" steps:

1. Frame a decision that has two options (Yes or No).
2. Divide an area into two competing halves: a "Pro" side and "Con" side.
3. 

Notice how providing personal family background information changes the entire dynamic of the information used in the decision framework and how it influences the recommended decision. The decision process is more specific not only to the scenario, but also it provides contextual background information. This makes the decision process more personalized and potentially much more accurate! 

### Step 5 - Open Source AI Models with Commercial AI Providers  

Semantic Kernel can include mutliple AI service providers. This allows for hybrid workflows from a single Semantic Kernel instance like: 
* Capability Optimizations: Use SLMs for domain specific tasks and LLMs for complex decision reasoning
* Decision Optimizations: Apply an (ensemble) decision self-consitency pattern  
* Capacity Optimizations: Splitting functions, plugins, personas or agents across different AI services

In [22]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.4"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.4"
#r "nuget: Microsoft.SemanticKernel, 1.48.0"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// Retrieve the configuration settings for the Azure OpenAI service
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];

// Example to build a Kernel with Azure OpenAI and Opensource AI
#pragma warning disable SKEXP0010 
var semanticKernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        modelId: azureOpenAIModelDeploymentName,
        deploymentName: azureOpenAIModelDeploymentName,
        endpoint: azureOpenAIEndpoint,
        apiKey: azureOpenAIAPIKey)
    .AddOpenAIChatCompletion(
        modelId: "Phi-4",
        endpoint: new Uri("http://localhost:1234/v1/"),
        apiKey: "LMStudio")
    .Build();
#pragma warning restore SKEXP0010

Installed Packages Microsoft.Extensions.Configuration, 9.0.4 Microsoft.Extensions.Configuration.Json, 9.0.4 Microsoft.SemanticKernel, 1.48.0